In [9]:
import hopsworks
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import getpass

In [10]:
api_key = getpass.getpass(prompt="Enter your Hopsworks API key: ")

In [11]:
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/218261
Connected. Call `.close()` to terminate connection gracefully.


In [12]:
wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(
    name="wine",
    version=1,
    description="Read from Wine Quality dataset",
    labels=["quality"],
    query=query,
)

In [13]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (2.20s) 


RandomForestRegressor()

In [14]:
y_pred = rf.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RF RMSE: {rmse}")

RF RMSE: 0.759878799200033


In [16]:
mr = project.get_model_registry()

model_dir = "wine_model"

if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(rf, model_dir + "/wine_model.pkl")

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

wine_model = mr.python.create_model(
    name="wine_model",
    metrics={"accuracy": rmse},
    model_schema=model_schema,
    description="Wine Predictor",
)

wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 14315729/14315729 elapsed<00:15 remaining<00:0001it/s]
Uploading: 100.000%|██████████| 501/501 elapsed<00:01 remaining<00:00<00:36,  9.09s/it]
Model export complete: 100%|██████████| 6/6 [00:22<00:00,  3.71s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/218261/models/wine_model/1


Model(name: 'wine_model', version: 1)